In [227]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


In [228]:
# 1. Membaca data
data = pd.read_csv('Book1.csv', delimiter=';')  # Ganti 'dataset.csv' dengan nama file dataset yang sesuai

print (data)
print(data.columns)


        ID user_id food_item  jumlah tanggal_beli  stock like
0    75836    ibet     Apple       2   01/05/2023      4  0,8
1    75836    ibet    Banana       4   15/04/2023      9  0,8
2    75836    ibet    Orange       1   19/05/2023      7  0,6
3    75836    ibet    Tomato       3   28/04/2023      8  0,4
4    75836    ibet     Bread       2   10/05/2023      7  0,9
..     ...     ...       ...     ...          ...    ...  ...
196  34256    adam      Eggs       2   16/05/2023      6    1
197  34256    adam      Milk       1   24/04/2023     12  0,9
198  34256    adam      Rice       2   08/05/2023      4  0,7
199  34256    adam    Banana       1   21/04/2023      5    1
200  34256    adam     Pasta       3   13/05/2023      4  0,8

[201 rows x 7 columns]
Index(['ID', 'user_id', 'food_item', 'jumlah', 'tanggal_beli', 'stock',
       'like'],
      dtype='object')


In [229]:
# fitur
X = data[['user_id','food_item','jumlah','tanggal_beli','stock','like']]
print (X)


    user_id food_item  jumlah tanggal_beli  stock like
0      ibet     Apple       2   01/05/2023      4  0,8
1      ibet    Banana       4   15/04/2023      9  0,8
2      ibet    Orange       1   19/05/2023      7  0,6
3      ibet    Tomato       3   28/04/2023      8  0,4
4      ibet     Bread       2   10/05/2023      7  0,9
..      ...       ...     ...          ...    ...  ...
196    adam      Eggs       2   16/05/2023      6    1
197    adam      Milk       1   24/04/2023     12  0,9
198    adam      Rice       2   08/05/2023      4  0,7
199    adam    Banana       1   21/04/2023      5    1
200    adam     Pasta       3   13/05/2023      4  0,8

[201 rows x 6 columns]


In [230]:
#membuat user matriks 
user_item_matrix = data.pivot_table(index='ID', columns='food_item', values='jumlah')
print(user_item_matrix)


food_item  Apple  Banana  Bread  Chicken  Eggs  Milk  Orange  Pasta  Rice   
ID                                                                          
12019          2       1      3        1     2     1       1      3     2  \
12321          2       1      3        1     2     1       1      3     2   
18393          2       1      2        3     2     1       1      3     2   
20198          2       1      3        1     2     1       1      3     2   
20392          2       1      3        1     2     1       1      3     2   
33452          2       1      3        1     2     1       1      3     2   
34256          2       1      3        1     2     1       1      3     2   
62845          2       1      2        3     2     1       1      3     2   
65836          2       2      3        1     2     1       1      1     3   
69326          2       1      3        1     2     1       1      3     2   
72548          2       2      3        1     2     1       1      1     3   

In [231]:
#matriks user item

item_similarity_matrix = cosine_similarity(user_item_matrix.T)

print(item_similarity_matrix)

[[1.         0.8302842  0.98692754 0.87588714 0.98701299 0.99381202
  0.95049691 0.96144149 0.98213711 0.96034152]
 [0.8302842  1.         0.78943812 0.75125128 0.81400412 0.86248336
  0.8936332  0.84366149 0.90872368 0.90288885]
 [0.98692754 0.78943812 1.         0.78962036 0.97752823 0.97746723
  0.92869087 0.93622145 0.96468246 0.96533911]
 [0.87588714 0.75125128 0.78962036 1.         0.8605207  0.8743829
  0.84348234 0.85836557 0.85772594 0.7575276 ]
 [0.98701299 0.81400412 0.97752823 0.8605207  1.         0.99381202
  0.97425934 0.93522036 0.970445   0.94700344]
 [0.99381202 0.86248336 0.97746723 0.8743829  0.99381202 1.
  0.97913005 0.96039208 0.98648766 0.96833426]
 [0.95049691 0.8936332  0.92869087 0.84348234 0.97425934 0.97913005
  1.         0.92755488 0.96269074 0.95178588]
 [0.96144149 0.84366149 0.93622145 0.85836557 0.93522036 0.96039208
  0.92755488 1.         0.93639848 0.9515245 ]
 [0.98213711 0.90872368 0.96468246 0.85772594 0.970445   0.98648766
  0.96269074 0.936398

In [232]:
# Mendapatkan ID pengguna yang akan diberikan rekomendasi
target_user_id = 18393



In [233]:
# Mendapatkan riwayat interaksi pengguna
user_interactions = user_item_matrix.loc[target_user_id]
print (user_interactions)


food_item
Apple      2
Banana     1
Bread      2
Chicken    3
Eggs       2
Milk       1
Orange     1
Pasta      3
Rice       2
Tomato     1
Name: 18393, dtype: int64


In [234]:
# Mendapatkan item-item yang belum diinteraksikan oleh pengguna
uninteracted_items = user_interactions[user_interactions.isnull()].index

print (uninteracted_items)



Index([], dtype='object', name='food_item')


In [235]:
# Menghitung skor prediksi untuk item-item yang belum diinteraksikan
item_scores = item_similarity_matrix.T.dot(user_interactions.fillna(0))
print (item_scores)


[17.15907582 15.12864395 16.69673722 15.56924906 16.98627453 17.21743045
 16.76993649 16.79679321 17.04746608 16.6408567 ]


In [236]:
# Membuat dataframe hasil rekomendasi
recommendations = pd.DataFrame(item_scores, index=user_item_matrix.columns, columns=['like'])
recommendations = recommendations.loc[uninteracted_items].sort_values(by='like', ascending=False)

# Menampilkan daftar item-item rekomendasi
print(recommendations)

Empty DataFrame
Columns: [like]
Index: []
